In [1]:
%%capture
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import networkx as nx
import pandas as pd
import sys
sys.path.append('/home/wrwt/Programming/pygraphmodels')
import graphmodels as gm
from itertools import permutations
from graphmodels import MatrixGraph, DGM
%matplotlib inline
%load_ext line_profiler

In [2]:
x = np.random.randint(0, 2, size=10000)
y = np.random.randint(0, 2, size=10000)

In [3]:
df = pd.DataFrame(data={'x': x, 'y': y})

In [4]:
gm.information.discrete_entropy(df[['x']])

0.69308667934004498

In [5]:
gm.information.discrete_entropy(df[['y']])

0.6931151802187312

In [6]:
gm.information.discrete_entropy(df[['x', 'y']])

1.3862016228433198

In [7]:
2 * -0.5 * np.log(0.5)

0.69314718055994529

In [8]:
4 * -0.5 * np.log(0.5)

1.3862943611198906

In [9]:
from os import listdir
import os.path
NETWORKS_PATH = '/home/wrwt/Programming/pygraphmodels/networks/'
network_filenames = listdir(NETWORKS_PATH)
true_dgm = gm.DGM.read(os.path.join(NETWORKS_PATH, 'earthquake.bif'))
true_dgm.draw()

In [10]:
data = true_dgm.rvs(size=100000)

In [11]:
gm.information.discrete_entropy(data[['Earthquake', 'Burglary', 'Alarm', 'MaryCalls']])

0.24307928568632489

In [12]:
gm.information.discrete_entropy(data[['Earthquake', 'Burglary']])

0.15630521173447165

In [13]:
gm.information.discrete_entropy(data[['Earthquake']]) + gm.information.discrete_entropy(data[['Burglary']])

0.15630520496575187

In [163]:
from itertools import repeat
class SubsetGraph(nx.DiGraph):
    def __init__(self):
        nx.DiGraph.__init__(self)
        
    def add_subset(self, subs, f=None):
        subs = frozenset(subs)
        if subs in self.nodes():
            return 
        self.add_node(subs, f=f)
        for node in list(self.nodes()):
            if node > subs and all(not child > subs for child in self.successors(node)):
                self.add_edge(node, subs)
        
        for pa, ch in list(self.edges()):
            if subs < pa and ch < subs:
                self.remove_edge(pa, ch)
                self.add_edge(subs, ch)
        for ch in list(self.nodes()):
            if ch < subs and all(not parent <= subs for parent in self.predecessors(ch)):
                self.add_edge(subs, ch)
                
    def get_parents(self, subs):
        subs = frozenset(subs)
        if subs in self.nodes():
            return self.predecessors(subs)
        results = []
        for node in list(self.nodes()):
            if node > subs and all(not child > subs for child in self.successors(node)):
                results.append(node)
        return results
    
    def get_children(self, subs):
        subs = frozenset(subs)
        if subs in self.nodes():
            return self.successors(subs)
        results = set()
        for pa, ch in list(self.edges()):
            if subs < pa and ch < subs:
                results.add(ch)
        for ch in list(self.nodes()):
            if ch < subs and all(not parent <= subs for parent in self.predecessors(ch)):
                results.add(ch)
        return results
    
    def upper_bound(self, subs):
        #print 'calculating upper_bound of', subs
        subs = frozenset(subs)
        if subs in self.nodes():
            return self.node[subs]['f']
        estimates = []
        for ch in self.get_children(subs):
            estimates.append(self.node[ch]['f'] + self.upper_bound(subs - ch))
        #return np.inf
        #return np.min(estimates)
        return min(np.min(estimates), np.min(self.node[pa]['f'] for pa in self.get_parents(subs)))
    
    def lower_bound1(self, subs):
        #print 'calculating lower_bound of', subs
        subs = frozenset(subs)
        if subs in self.nodes():
            return self.node[subs]['f']
        estimates = [0]
        for ch in self.get_children(subs):
            estimates.append(max(self.node[ch]['f'], self.lower_bound(subs - ch)))
        return np.max(estimates)
    
    def lower_bound2(self, subs):
        #print 'calculating lower_bound of', subs
        subs = frozenset(subs)
        if subs in self.nodes():
            return self.node[subs]['f']
        estimates = [0]
        for pa in self.get_parents(subs):
            estimates.append(self.node[pa]['f'] - self.upper_bound(pa - subs))
        return np.max(estimates)
    
    def lower_bound(self, subs):
        #return 0
        #return self.lower_bound1(subs)
        return max(self.lower_bound1(subs), self.lower_bound2(subs))

In [120]:
sg = SubsetGraph()
sg.add_subset([1], 0.5)
sg.add_subset([3, 4], 1.1)
sg.add_subset([1, 2], 0.7)
sg.add_subset([2], 0.5)
sg.add_subset([3], 0.3)
sg.add_subset([4], 1.0)
sg.add_subset([1, 2, 3], 0.9)
sg.add_subset([1, 2, 3, 4], 1.9)
sg.lower_bound([1, 3, 4]), sg.upper_bound([1, 3, 4])

(0, 1.6000000000000001)

In [121]:
sg = SubsetGraph()
sg.add_subset([1], 0.3)
sg.add_subset([2], 0.4)
sg.add_subset([3], 0.5)
sg.add_subset([4], 0.6)
sg.add_subset([1, 2], 0.55)
sg.add_subset([2, 3], 0.7)
sg.add_subset([3, 4], 1.0)
sg.lower_bound([1, 4]), sg.upper_bound([1, 4])

(0, 0.89999999999999991)

In [122]:
data = true_dgm.rvs(10000)

In [123]:
sg = SubsetGraph()
ee = gm.EntropyEstimator(gm.MatrixGraph.from_networkx_DiGraph(true_dgm, order=data.columns), data)

In [124]:
def add_subs(*subs):
    subs = frozenset(subs)
    f = ee([name in subs for name in data.columns])
    sg.add_subset(subs, f)

In [125]:
def entr(*subs):
    return ee([name in subs for name in data.columns])

In [126]:
def bounds(*subs):
    return sg.lower_bound(frozenset(subs)), sg.upper_bound(frozenset(subs))

In [127]:
for var in data.columns:
    add_subs(var)

In [128]:
add_subs('Burglary', 'Earthquake', 'MaryCalls', 'JohnCalls')

In [129]:
%time entr('Burglary', 'MaryCalls')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 43.9 µs


0

In [130]:
#bounds('Alarm', 'Burglary')

In [131]:
#bounds('Alarm', 'Earthquake')

In [132]:
#bounds('Alarm', 'Earthquake', 'MaryCalls')[1] - bounds('Alarm', 'Earthquake')[0]

In [133]:
#bounds('Alarm', 'Earthquake', 'MaryCalls')[0] - bounds('Alarm', 'Earthquake')[1]

In [134]:
import numpy as np
from graphmodels.information import discrete_entropy


class EntropyEstimator:
    def __init__(self, graph, data):
        self.graph = graph
        self.data = data
        self.cache = {}

    def _footprint(self, nodes):
        return tuple(nodes)

    def __call__(self, nodes):
        fp = self._footprint(nodes)
        if fp in self.cache:
            return self.cache[fp]

        if np.any(nodes):
            subdata = self.data[self.data.columns[nodes]]
            result = discrete_entropy(subdata)
        else:
            result = 0

        self.cache[fp] = result
        return result


class InformationEstimator:
    def __init__(self, graph, data):
        self.graph = graph
        self.entropy_estimator = EntropyEstimator(graph, data)
        self.chosen = []

    def __call__(self, node, parents):
        parents = np.array(parents, dtype=bool)
        h1 = self.entropy_estimator(parents)
        temp = np.zeros(len(parents), dtype=bool)
        temp[node] = True
        h2 = self.entropy_estimator(temp)
        parents[node] = True
        h12 = self.entropy_estimator(parents)
        return h1 + h2 - h12


class ScoreBIC:
    def __init__(self, graph, data):
        self.graph = graph
        self.data = data
        self.n_values = np.asarray([len(self.data[column].value_counts()) for column in self.data.columns])
        self.mi_estimator = InformationEstimator(graph, data)

    def __call__(self, node, parents):
        parents = np.asarray(parents, dtype=bool)
        k = self.n_values[node]*np.prod(self.n_values[parents]) - 1
        n = self.data.shape[0]
        l = n*self.mi_estimator(node, parents)

        result = l - 0.5 * np.log(n) * k
        return result

    def total(self):
        score = 0.
        for node in self.graph.nodes():
            pa = self.graph.adj[:, node].copy()
            score += self(node, pa)
        return score

In [142]:
class AdvancedEntropyEstimator:
    def __init__(self, graph, data):
        self.graph = graph
        self.data = data
        self.sg = SubsetGraph()
        self._initialize_sg()

    def _initialize_sg(self):
        vec = np.asarray([False] * len(self.data.columns), dtype=bool)
        for i, column in enumerate(self.data.columns):
            vec[i] = True
            self.__call__(vec)
            vec[i] = False
        
    def _key(self, subs):
        return frozenset([name for name, included in zip(data.columns, subs) if included])
    
    def _add_subs(self, subs, f):
        self.sg.add_subset(subs, f)
    
    def __call__(self, nodes):
        if not np.any(nodes):
            return 0
        key = self._key(nodes)
        if key in self.sg.nodes():
            return self.sg.node[key]['f']
        subdata = self.data[self.data.columns[nodes]]
        result = discrete_entropy(subdata)
        self._add_subs(key, f=result)
        return result
    
    def lower_bound(self, nodes):
        if not np.any(nodes):
            return 0
        key = self._key(nodes)
        return self.sg.lower_bound(key)
    
    def upper_bound(self, nodes):
        if not np.any(nodes):
            return 0
        key = self._key(nodes)
        return self.sg.upper_bound(key)
    
class AdvancedInformationEstimator:
    def __init__(self, graph, data):
        self.graph = graph
        self.entropy_estimator = AdvancedEntropyEstimator(graph, data)
        self.chosen = []

    def __call__(self, node, parents):
        parents = np.array(parents, dtype=bool)
        h1 = self.entropy_estimator(parents)
        temp = np.zeros(len(parents), dtype=bool)
        temp[node] = True
        h2 = self.entropy_estimator(temp)
        parents[node] = True
        h12 = self.entropy_estimator(parents)
        return h1 + h2 - h12
    
    def lower_bound(self, node, parents):
        parents = np.array(parents, dtype=bool)
        h1 = self.entropy_estimator.lower_bound(parents)
        temp = np.zeros(len(parents), dtype=bool)
        temp[node] = True
        h2 = self.entropy_estimator.lower_bound(temp)
        parents[node] = True
        h12 = self.entropy_estimator.upper_bound(parents)
        return h1 + h2 - h12
        
    def upper_bound(self, node, parents):
        parents = np.array(parents, dtype=bool)
        h1 = self.entropy_estimator.upper_bound(parents)
        temp = np.zeros(len(parents), dtype=bool)
        temp[node] = True
        h2 = self.entropy_estimator.upper_bound(temp)
        parents[node] = True
        h12 = self.entropy_estimator.lower_bound(parents)
        return h1 + h2 - h12
    
class AdvancedScoreBIC:
    def __init__(self, graph, data):
        self.graph = graph
        self.data = data
        self.n_values = np.asarray([len(self.data[column].value_counts()) for column in self.data.columns])
        self.mi_estimator = AdvancedInformationEstimator(graph, data)

    def __call__(self, node, parents, option=None):
        parents = np.asarray(parents, dtype=bool)
        k = self.n_values[node]*np.prod(self.n_values[parents]) - 1
        n = self.data.shape[0]
        
        if option == 'lower_bound':
            l = n*self.mi_estimator.lower_bound(node, parents)
        elif option == 'upper_bound':
            l = n*self.mi_estimator.upper_bound(node, parents)
        else:
            l = n*self.mi_estimator(node, parents)

        result = l - 0.5 * np.log(n) * k
        return result
    
    def lower_bound(self, node, parents):
        parents = np.asarray(parents, dtype=bool)
        k = self.n_values[node]*np.prod(self.n_values[parents]) - 1
        n = self.data.shape[0]
        l = n*self.mi_estimator.lower_bound(node, parents)

        result = l - 0.5 * np.log(n) * k
        return result

    def upper_bound(self, node, parents):
        parents = np.asarray(parents, dtype=bool)
        k = self.n_values[node]*np.prod(self.n_values[parents]) - 1
        n = self.data.shape[0]
        l = n*self.mi_estimator.upper_bound(node, parents)

        result = l - 0.5 * np.log(n) * k
        return result
    
    def total(self):
        score = 0.
        for node in self.graph.nodes():
            pa = self.graph.adj[:, node].copy()
            score += self(node, pa)
        return score

In [158]:
from graphmodels import LocalOperation

class AdvancedAddEdge(LocalOperation):
    def __init__(self, graph, fscore, src, dst):
        LocalOperation.__init__(self, graph, fscore)
        self.src = src
        self.dst = dst

    def do(self):
        if self.graph.adj[self.src, self.dst]:
            raise InvalidOperation()
        self.graph.adj[self.src, self.dst] = 1
        if not self.graph.is_acyclic():
            self.graph.adj[self.src, self.dst] = 0
            raise InvalidOperation()
        return self

    def undo(self):
        self.graph.adj[self.src, self.dst] = 0
        return self

    def score(self):
        pa = self.graph.adj[:, self.dst].copy()
        score = -self.fscore(self.dst, pa)
        pa[self.src] = 1
        score += self.fscore(self.dst, pa)
        return score
    
    def lower_bound(self):
        pa = self.graph.adj[:, self.dst].copy()
        score = -self.fscore.upper_bound(self.dst, pa)
        pa[self.src] = 1
        score += self.fscore.lower_bound(self.dst, pa)
        return score
    
    def upper_bound(self):
        pa = self.graph.adj[:, self.dst].copy()
        score = -self.fscore.lower_bound(self.dst, pa)
        pa[self.src] = 1
        score += self.fscore.upper_bound(self.dst, pa)
        return score
    


class AdvancedRemoveEdge(LocalOperation):
    def __init__(self, graph, fscore, src, dst):
        LocalOperation.__init__(self, graph, fscore)
        self.src = src
        self.dst = dst

    def do(self):
        if not self.graph.adj[self.src, self.dst]:
            raise InvalidOperation()
        self.graph.adj[self.src, self.dst] = 0
        return self

    def undo(self):
        self.graph.adj[self.src, self.dst] = 1
        return self

    def score(self):
        pa = self.graph.adj[:, self.dst].copy()
        score = -self.fscore(self.dst, pa)
        pa[self.src] = 0
        score += self.fscore(self.dst, pa)
        return score
    
    def lower_bound(self):
        pa = self.graph.adj[:, self.dst].copy()
        score = -self.fscore.upper_bound(self.dst, pa)
        pa[self.src] = 0
        score += self.fscore.lower_bound(self.dst, pa)
        return score

    def upper_bound(self):
        pa = self.graph.adj[:, self.dst].copy()
        score = -self.fscore.lower_bound(self.dst, pa)
        pa[self.src] = 0
        score += self.fscore.upper_bound(self.dst, pa)
        return score

class AdvancedReverseEdge(LocalOperation):
    def __init__(self, graph, fscore, src, dst):
        LocalOperation.__init__(self, graph, fscore)
        self.src = src
        self.dst = dst

    def do(self):
        if not self.graph.adj[self.src, self.dst]:
            raise InvalidOperation()
        self.graph.adj[self.src, self.dst] = 0
        self.graph.adj[self.dst, self.src] = 1
        if not self.graph.is_acyclic():
            self.graph.adj[self.src, self.dst] = 1
            self.graph.adj[self.dst, self.src] = 0
            raise InvalidOperation()
        return self

    def undo(self):
        self.graph.adj[self.src, self.dst] = 1
        self.graph.adj[self.dst, self.src] = 0
        return self

    def score(self):
        pa = self.graph.adj[:, self.dst].copy()
        score = -self.fscore(self.dst, pa)
        pa[self.src] = 0
        score += self.fscore(self.dst, pa)

        pa = self.graph.adj[:, self.src].copy()
        score -= self.fscore(self.src, pa)
        pa[self.dst] = 1
        score += self.fscore(self.src, pa)
        return score
    
    def lower_bound(self):
        pa = self.graph.adj[:, self.dst].copy()
        score = -self.fscore.upper_bound(self.dst, pa)
        pa[self.src] = 0
        score += self.fscore.lower_bound(self.dst, pa)

        pa = self.graph.adj[:, self.src].copy()
        score -= self.fscore.upper_bound(self.src, pa)
        pa[self.dst] = 1
        score += self.fscore.lower_bound(self.src, pa)
        return score
    
    def upper_bound(self):
        pa = self.graph.adj[:, self.dst].copy()
        score = -self.fscore.lower_bound(self.dst, pa)
        pa[self.src] = 0
        score += self.fscore.upper_bound(self.dst, pa)

        pa = self.graph.adj[:, self.src].copy()
        score -= self.fscore.lower_bound(self.src, pa)
        pa[self.dst] = 1
        score += self.fscore.upper_bound(self.src, pa)
        return score

In [160]:
from graphmodels import AddEdge, RemoveEdge, ReverseEdge, InvalidOperation
class AdvancedGreedySearch:
    def __init__(self, data, cls_score):
        graph = nx.DiGraph()
        graph.add_nodes_from(data.columns)
        graph = MatrixGraph.from_networkx_DiGraph(graph, order=data.columns)
        self.graph = graph
        self.fscore = cls_score(graph, data)
        self.tscore = ScoreBIC(graph, data)

        self.ops = []
        self.ops += [AdvancedAddEdge(graph, self.fscore, u, v) for u, v in permutations(graph.nodes(), 2)]
        self.ops += [AdvancedRemoveEdge(graph, self.fscore, u, v) for u, v in permutations(graph.nodes(), 2)]
        self.ops += [AdvancedReverseEdge(graph, self.fscore, u, v) for u, v in permutations(graph.nodes(), 2)]
    
    def iteration(self):
        #ops.sort(reverse=True, key=lambda x: x[1] + x[2])
        
        best_op = None
        best_score = 0.
        
        skipped = 0
        
        for op in self.ops:
            ub = op.upper_bound()
            if ub < best_score:
                skipped += 1
                continue
            try:
                current_score = op.score()
                op.do()
                if current_score > best_score:
                    best_score = current_score
                    best_op = op
                op.undo()
            except InvalidOperation:
                pass
        
        print 'done {}, skippped {} operations out of {}'.format(len(self.ops) - skipped, skipped, len(self.ops))
        if best_op is None or best_op.score() < 1e-5:
            return True
        best_op.do()
        return False

    def __call__(self, max_iter=40, verbose=True):
        counter = 0
        while not self.iteration() and counter < max_iter:
            if verbose:
                print(self.fscore.total())
            counter += 1
        return DGM(self.graph.to_networkx_DiGraph())

In [137]:
from os import listdir
import os.path
NETWORKS_PATH = '/home/wrwt/Programming/pygraphmodels/networks/'
network_filenames = listdir(NETWORKS_PATH)
true_dgm = gm.DGM.read(os.path.join(NETWORKS_PATH, 'alarm.bif'))
true_dgm.draw()

In [138]:
data = true_dgm.rvs(size=100000)

In [164]:
gs = AdvancedGreedySearch(data, AdvancedScoreBIC)

In [165]:
%%time
res = gs(max_iter=100)

done 297, skippped 3699 operations out of 3996
60960.1071108
done 86, skippped 3910 operations out of 3996
113855.940948
done 24, skippped 3972 operations out of 3996
166110.820999
done 13, skippped 3983 operations out of 3996
217414.634911
done 12, skippped 3984 operations out of 3996
264464.797122
done 13, skippped 3983 operations out of 3996
309697.293756
done 11, skippped 3985 operations out of 3996
354480.061847
done 51, skippped 3945 operations out of 3996
397479.067507
done 26, skippped 3970 operations out of 3996
437509.736362
done 19, skippped 3977 operations out of 3996
473496.056222
done 20, skippped 3976 operations out of 3996
508389.586626
done 90, skippped 3906 operations out of 3996
539983.400972
done 77, skippped 3919 operations out of 3996
571237.987646
done 52, skippped 3944 operations out of 3996
602562.244423
done 28, skippped 3968 operations out of 3996
633148.088215
done 36, skippped 3960 operations out of 3996
661303.498134
done 35, skippped 3961 operations out o

In [149]:
from graphmodels import GreedySearch
gs = GreedySearch(data, ScoreBIC)

In [150]:
%%time
res = gs(max_iter=100)

60960.1071108
113855.940948
166110.820999
217414.634911
264464.797122
309697.293756
354480.061847
397479.067507
437509.736362
473496.056222
508389.586626
539983.400972
571237.987646
602562.244423
633148.088215
661303.498134
687734.598457
713414.653196
738449.399538
763080.131291
780610.959469
800606.418292
817276.946688
832514.905447
847183.9627
861524.743612
875430.788913
888540.251601
900196.789865
911214.945808
921842.689448
930951.691249
939116.415423
946043.657482
952441.167711
957989.13935
962314.19845
966529.163755
970487.295592
974335.405936
977656.166694
980720.721297
983771.565871
986391.318953
988548.128189
990630.577411
992451.543266
993783.78331
994899.468299
995950.685317
996232.72998
996423.287356
996549.33501
996641.438414
996735.469207
996786.296945
CPU times: user 2min 31s, sys: 48 ms, total: 2min 31s
Wall time: 2min 31s


In [96]:
gs.fscore.mi_estimator.entropy_estimator.sg.nodes(data=True)

AttributeError: EntropyEstimator instance has no attribute 'sg'

In [ ]:
data.columns

In [ ]:
vec = [False] * len(data.columns)
#vec[list(data.columns).index('CVP')] = True
vec[list(data.columns).index('LVEDVOLUME')] = True
vec[8] = True
vec[9] = True
vec[11] = True
vec[20] = True

In [ ]:
%%time
gs.fscore.lower_bound(list(data.columns).index('CVP'), vec)

In [ ]:
gs.fscore.upper_bound(list(data.columns).index('CVP'), vec)

In [ ]:
%%time
gs.fscore(list(data.columns).index('CVP'), vec)

In [ ]:
len(gs.fscore.mi_estimator.entropy_estimator.sg.nodes())